In [1]:
# AXILiteS
# 0x0000 : Control signals
#          bit 0  - ap_start (Read/Write/COH)
#          bit 1  - ap_done (Read/COR)
#          bit 2  - ap_idle (Read)
#          bit 3  - ap_ready (Read)
#          bit 7  - auto_restart (Read/Write)
#          others - reserved
# 0x0004 : Global Interrupt Enable Register
#          bit 0  - Global Interrupt Enable (Read/Write)
#          others - reserved
# 0x0008 : IP Interrupt Enable Register (Read/Write)
#          bit 0  - Channel 0 (ap_done)
#          bit 1  - Channel 1 (ap_ready)
#          others - reserved
# 0x000c : IP Interrupt Status Register (Read/TOW)
#          bit 0  - Channel 0 (ap_done)
#          bit 1  - Channel 1 (ap_ready)
#          others - reserved
# 0x0c00 : Data signal of num_symbols
#          bit 31~0 - num_symbols[31:0] (Read/Write)
# 0x0c04 : reserved
# 0x0400 ~
# 0x07ff : Memory 'in_value_V' (256 * 32b)
#          Word n : bit [31:0] - in_value_V[n]
# 0x0800 ~
# 0x0bff : Memory 'in_frequency_V' (256 * 32b)
#          Word n : bit [31:0] - in_frequency_V[n]
# 0x1000 ~
# 0x13ff : Memory 'parent_V' (255 * 32b)
#          Word n : bit [31:0] - parent_V[n]
# 0x1400 ~
# 0x17ff : Memory 'left_V' (255 * 32b)
#          Word n : bit [31:0] - left_V[n]
# 0x1800 ~
# 0x1bff : Memory 'right_V' (255 * 32b)
#          Word n : bit [31:0] - right_V[n]
# (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)

from pynq import Overlay

overlay = Overlay("../bitstream/create-tree-test.bit")
overlay.download()

create_tree = overlay.create_tree_0

# Sorted Table
# ------------
#  F   |  1
#  B   |  1
#  C   |  2
#  A   |  3
#  D   |  5
#  E   |  5

sorted_table = [
    { 'symbol': 'f', 'freq': 1 },
    { 'symbol': 'b', 'freq': 1 },
    { 'symbol': 'c', 'freq': 2 },
    { 'symbol': 'a', 'freq': 3 },
    { 'symbol': 'd', 'freq': 5 },
    { 'symbol': 'e', 'freq': 5 }
]

# set symbol numbers
create_tree.write(0x0c00, 6)

# write register
for idx, pair in enumerate(sorted_table):
    # write symbol order
    create_tree.write(0x0400 + 4*idx, ord(pair['symbol']))
    # write symbol frequency
    create_tree.write(0x0800 + 4*idx, pair['freq'])

# start
create_tree.write(0x0000, 1)

from time import sleep

sleep(5)

# read created tree
for i in range(10):
    # read node
    parent = create_tree.read(0x1000 + 4*i)
    lchild = create_tree.read(0x1400 + 4*i)
    rchild = create_tree.read(0x1800 + 4*i)
    # read left & right
    print("pos : " + str(i) + " parent : " + str(parent))
    print("        lchild : " + str(lchild))
    print("        rchild : " + str(rchild))

pos : 0 parent : 1
        lchild : 102
        rchild : 98
pos : 1 parent : 2
        lchild : 99
        rchild : 4294967295
pos : 2 parent : 4
        lchild : 97
        rchild : 4294967295
pos : 3 parent : 4
        lchild : 100
        rchild : 101
pos : 4 parent : 0
        lchild : 4294967295
        rchild : 4294967295
pos : 5 parent : 0
        lchild : 0
        rchild : 0
pos : 6 parent : 0
        lchild : 0
        rchild : 0
pos : 7 parent : 0
        lchild : 0
        rchild : 0
pos : 8 parent : 0
        lchild : 0
        rchild : 0
pos : 9 parent : 0
        lchild : 0
        rchild : 0
